# Analysis of Test Durations: Xsens (.txt) vs TDMS (.tdms)

Goal: Iterate through all available Xsens data files, find the matching TDMS file, calculate the duration of each, and compare them to check for inconsistencies. Also display Start Times (Timeonly).

In [5]:
import pandas as pd
import glob
import os
from nptdms import TdmsFile
from datetime import timedelta

# Define the root directory for data
data_dir = '../'

# Search for Xsens files
file_pattern = os.path.join(data_dir, 'Moto_*_P*.txt')
files = glob.glob(file_pattern)

results = []

print(f"Found {len(files)} Xsens files to analyze.")

for file_path in files:
    try:
        filename = os.path.basename(file_path)
        # Construct matching TDMS filename
        base_no_p = filename.split('_P')[0]
        tdms_candidate_1 = os.path.join(data_dir, base_no_p + '.tdms')
        tdms_candidate_2 = file_path.replace('.txt', '.tdms')
        
        tdms_path = None
        if os.path.exists(tdms_candidate_1):
            tdms_path = tdms_candidate_1
        elif os.path.exists(tdms_candidate_2):
            tdms_path = tdms_candidate_2
            
        # Parse filename for metadata
        parts = filename.replace('.txt', '').split('_')
        speed = 'N/A'
        passage = 'N/A'
        for p in parts:
            if p.startswith('P') and p[1:].isdigit():
                passage = p
            elif p.isdigit():
                speed = p
        
        # 1. Calculate Xsens Duration & Start Time
        xsens_duration_sec = None
        xsens_start_time = None
        
        header_df = pd.read_csv(file_path, sep='\t', skiprows=12,  nrows=1)
        cols = header_df.columns.str.strip().tolist()
        req_cols = ['UTC_Year', 'UTC_Month', 'UTC_Day', 'UTC_Hour', 'UTC_Minute', 'UTC_Second', 'UTC_Nano']
        
        if all(c in cols for c in req_cols):
            df = pd.read_csv(file_path, sep='\t', skiprows=12, usecols=req_cols)
            df.columns = df.columns.str.strip()
            df = df.dropna(subset=req_cols)
            
            if not df.empty:
                def get_time(row):
                    return pd.Timestamp(year=int(row['UTC_Year']), month=int(row['UTC_Month']), day=int(row['UTC_Day']),
                                      hour=int(row['UTC_Hour']), minute=int(row['UTC_Minute']), second=int(row['UTC_Second']),
                                      microsecond=int(row['UTC_Nano'] / 1000))
                xsens_start_time = get_time(df.iloc[0])
                end_time = get_time(df.iloc[-1])
                xsens_duration_sec = (end_time - xsens_start_time).total_seconds()
        
        # 2. Calculate TDMS Duration & Start Time
        tdms_duration_sec = None
        tdms_start_time = None
        
        if tdms_path:
            try:
                with TdmsFile.open(tdms_path) as tdms_file:
                    for group in tdms_file.groups():
                        if len(group.channels()) > 0:
                            channel = group.channels()[0]
                            props = channel.properties
                            
                            samples = len(channel)
                            increment = props.get('wf_increment', 0.0025)
                            if increment == 0: increment = 0.0025
                            tdms_duration_sec = samples * increment
                            
                            if 'wf_start_time' in props:
                                tdms_start_time = props['wf_start_time']
                                tdms_start_time = pd.Timestamp(tdms_start_time).tz_localize(None) 
                            
                            break
            except Exception as e:
                print(f"  Error reading TDMS {tdms_path}: {e}")
        
        # 3. Store Results (Format Times)
        diff = None
        if xsens_duration_sec is not None and tdms_duration_sec is not None:
            diff = tdms_duration_sec - xsens_duration_sec
            
        # Helper to format time only
        def fmt_time(ts):
            if pd.isna(ts):
                return None
            return ts.strftime('%H:%M:%S.%f')
            
        results.append({
            'Condition': base_no_p,
            'Speed': speed,
            'Xsens_File': filename,
            'Xsens_Start': fmt_time(xsens_start_time),
            'TDMS_Start': fmt_time(tdms_start_time),
            'Xsens_Dur(s)': round(xsens_duration_sec, 3) if xsens_duration_sec else None,
            'TDMS_Dur(s)': round(tdms_duration_sec, 3) if tdms_duration_sec else None,
            'Diff(s)': round(diff, 3) if diff else None,
        })
        
        print(f"Processed {base_no_p}: Xsens Start={fmt_time(xsens_start_time)}")

    except Exception as e:
        print(f"  Error processing {file_path}: {e}")

# Create Summary DataFrame
summary_df = pd.DataFrame(results)

# Display
if not summary_df.empty:
    cols = ['Condition', 'Speed', 'Xsens_Start', 'TDMS_Start', 'Xsens_Dur(s)', 'TDMS_Dur(s)', 'Diff(s)']
    print("\n=== TEST TIMING SUMMARY (Time Only) ===\n")
    display(summary_df[cols].sort_values(by=['Condition']))
    
    summary_df.to_csv('Timing_Comparison.csv', index=False)
else:
    print("No data to analyze.")

Found 4 Xsens files to analyze.
Processed Moto_Chicane_100: Xsens Start=14:29:37.248402
Processed Moto_Chicane_50: Xsens Start=10:54:09.433662
Processed Moto_Chicane_mouille_80: Xsens Start=14:58:11.718675
Processed Moto_Freinage_mouille_50: Xsens Start=10:20:50.137677

=== TEST TIMING SUMMARY (Time Only) ===



,Condition,Speed,Xsens_Start,TDMS_Start,Xsens_Dur(s),TDMS_Dur(s),Diff(s)
0,Moto_Chicane_100,100,14:29:37.248402,14:29:36.687152,55.192,55.0,-0.192
1,Moto_Chicane_50,50,10:54:09.433662,10:50:39.214080,62.260,62.5,0.240
2,Moto_Chicane_mouille_80,80,14:58:11.718675,14:45:33.807814,55.007,110.0,54.993
3,Moto_Freinage_mouille_50,50,10:20:50.137677,10:20:49.787921,64.642,65.0,0.358
